In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers requests beautifulsoup4 pandas numpy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.7 MB/s eta 0:11:08tcmalloc: large alloc 1147494400 bytes == 0x55e10509a000 @  0x7ff231166615 0x55e0cbed33bc 0x55e0cbfb418a 0x55e0cbed61cd 0x55e0cbfc8b3d 0x55e0cbf4a458 0x55e0cbf4502f 0x55e0cbed7aba 0x55e0cbf4a2c0 0x55e0cbf4502f 0x55e0cbed7aba 0x55e0cbf46cd4 0x55e0cbfc9986 0x55e0cbf46350 0x55e0cbfc9986 0x55e0cbf46350 0x55e0cbfc9986 0x55e0cbf46350 0x55e0cbed7f19 0x55e0cbf1ba79 0x55e0cbed6b32 0x55e0cbf4a1dd 0x55e0cbf4502f 0x55e0cbed7aba 0x55e0cbf46cd4 0x55e0cbf4502f 0x55e0cbed7aba 0x55e0cbf45eae 0x55e0cbed79da 0x55e0cbf46108 0x55e0cbf4502f
     |█████████████████               | 1055.7 MB 1.5 MB/s eta 0:10:29tcmalloc: large alloc 1434370048 bytes == 0x55e1496f0000 @  0x7ff231166615 0x55e0cbed33bc 0x55e0cbfb418a 0x55e0cbed61cd 0x55e0cbfc8b3d 0x55e0cbf4a458 0x55e0cbf4502f 0x55e0cbed7aba 0x55e0cbf4a2c0 0x55e0cbf4502f 0x55e0cbed7aba 0x55e0cbf46cd4 0x55e0cbfc9986 0x55e0cbf463

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [11]:
tokens = tokenizer.encode('not that likely that it could have been that worst but it was Great', return_tensors='pt')
result = model(tokens)
int(torch.argmax(result.logits))+1


4

In [12]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [13]:
import numpy as np
import pandas as pd
df = pd.DataFrame(np.array(reviews), columns=['review'])

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df.head(5)

,review,sentiment
0,I went here a little while ago- a beautiful mo...,2
1,I came to Social brew cafe for brunch while ex...,5
2,Ricotta hot cakes! These were so yummy. I ate ...,5
3,We came for brunch twice in our week-long visi...,4
4,Good coffee and toasts. Straight up and down -...,5
